### Data cleaning & feature engineering

Tasks:
- Clean data, especially for the old events many values are missing
- get interpretable data types for prediction models
- add fighter stats for each fighter
- Fight statistics are available only after the fight, so each row must contain the cumulative statistics of all past fights of both participants and their respective opponents. Otherwise, it is not possible to make a prediction before the actual fight.

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [2]:
# fight data
df_fights = pd.read_csv('./data/fights.csv', delimiter=";")

# Join fighter data to red and blue fighter
df_fighters = pd.read_csv('./data/fighters.csv', delimiter=";")

df_fights = pd.merge(df_fights, df_fighters, left_on="FIGHTER_R", right_on="NAME", how="left", suffixes=("","_R"))
df_fights = pd.merge(df_fights, df_fighters, left_on="FIGHTER_R", right_on="NAME", how="left", suffixes=("","_R"))
df_fights = pd.merge(df_fights, df_fighters, left_on="FIGHTER_B", right_on="NAME", how="left", suffixes=('','_B'))

df_fights = df_fights.drop(list(df_fighters.columns), axis=1)

In [3]:
print(df_fights.groupby(["ROUNDS"])["WINNER"].count())

ROUNDS
1 Rnd (10)                 6
1 Rnd (12)                 4
1 Rnd (15)                 8
1 Rnd (18)                 2
1 Rnd (20)                21
1 Rnd (30)                 1
1 Rnd + 2OT (15-3-3)      20
1 Rnd + 2OT (24-3-3)       1
1 Rnd + OT (12-3)         80
1 Rnd + OT (15-3)          2
1 Rnd + OT (27-3)          1
1 Rnd + OT (30-3)          1
1 Rnd + OT (30-5)          3
1 Rnd + OT (31-5)          1
2 Rnd (5-5)               14
3 Rnd (5-5-5)           6080
3 Rnd + OT (5-5-5-5)      26
5 Rnd (5-5-5-5-5)        597
No Time Limit             29
Name: WINNER, dtype: int64


In [4]:
# In order to keep only fights with a similar and current set of rules, events that do not go over 3 or 5 rounds will be deleted. have 3 or 5 rounds. 
df_fights = df_fights[(df_fights["ROUNDS"] == "3 Rnd (5-5-5)") | (df_fights["ROUNDS"] == "5 Rnd (5-5-5-5-5)")]

# remove open weight fights
df_fights = df_fights[df_fights["BOUT"]!="Open Weight Bout"]

# replace missing time values with 00:00
df_fights[["CTRL_B", "CTRL_R", "LAST_ROUND_TIME"]] = df_fights[["CTRL_B", "CTRL_R", "LAST_ROUND_TIME"]].replace('--', '00:00')

In [5]:
# Convert Height and Reach to a usable number format

# height
df_fights["HEIGHT_R"] = df_fights["HEIGHT_R"].apply(lambda x: str(x).replace('"', "").replace("' ", ".") if x != '--' else np.nan).astype("float32")
df_fights["HEIGHT_B"] = df_fights["HEIGHT_B"].apply(lambda x: str(x).replace('"', "").replace("' ", ".") if x != '--' else np.nan).astype("float32")

# Reach
df_fights["REACH_R"] = df_fights["REACH_R"].apply(lambda x: str(x).replace('"', "") if x != '--' else np.nan).astype("float32")
df_fights["REACH_B"] = df_fights["REACH_B"].apply(lambda x: str(x).replace('"', "") if x != '--' else np.nan).astype("float32")

### Create new features "Title Fight"

In [6]:
# some event formats were unique and should not be included in a predictive model 
bouts = df_fights["BOUT"].value_counts() > 1
established_bouts = df_fights['BOUT'].value_counts()[bouts].index
df_fights = df_fights[df_fights["BOUT"].apply(lambda x: True if x in established_bouts else False)]

# Get a flag for each title fight
df_fights["TITLE_FIGHT"] = df_fights["BOUT"].apply(lambda x: 1 if 'Title Bout' in x else 0)

### Create new feature "Main Event"

- the main event is the last fight of a fight night

In [7]:
indexes_reversed = df_fights.index.tolist()
indexes_reversed.reverse()

main_event = []

for i in indexes_reversed:
    u = i-1
    earlier_fight = df_fights.loc[i]["ROUNDS"]
    try: 
        later_fight = df_fights.loc[i-1]["ROUNDS"]
    except:
        later_fight = '3 Rnd (5-5-5)'
    if later_fight != earlier_fight and earlier_fight == '5 Rnd (5-5-5-5-5)':
        main_event.append(1)
    else:
        main_event.append(0)

main_event.reverse()
df_fights['MAIN_EVENT'] = main_event

### Transform times into float-values and add new feature "Total Fight time"

In [8]:
from datetime import datetime

def time_to_minutes(time):
    if time is not None:
        clock_time = datetime.strptime(time,'%M:%S')
        minutes = clock_time.minute + clock_time.second/60
    else: 
        minutes = []
    return minutes

In [9]:
df_fights["CTRL_B"] = df_fights["CTRL_B"].apply(time_to_minutes)
df_fights["CTRL_R"] = df_fights["CTRL_R"].apply(time_to_minutes)
df_fights["LAST_ROUND_TIME"] = df_fights["LAST_ROUND_TIME"].apply(time_to_minutes)
# new feature total fight time will be summed 
df_fights["TOTAL_FIGHT_TIME"] = df_fights["LAST_ROUND"] + df_fights["LAST_ROUND_TIME"] - 1

# new feature avg fight time will be averaged
df_fights["AVG_FIGHT_TIME"] = df_fights["TOTAL_FIGHT_TIME"]

In [10]:
df_fights.columns

Index(['WINNER', 'BOUT', 'WIN_METHOD', 'LAST_ROUND', 'LAST_ROUND_TIME',
       'ROUNDS', 'REFEREE', 'FIGHTER_R', 'FIGHTER_B', 'KD_R', 'KD_B',
       'SIG_STR_R', 'SIG_STR_ATT_R', 'SIG_STR_B', 'SIG_STR_ATT_B',
       'TOTAL_STR_R', 'TOTAL_STR_ATT_R', 'TOTAL_STR_B', 'TOTAL_STR_ATT_B',
       'TAKEDOWN_R', 'TAKEDOWN_ATT_R', 'TAKEDOWN_B', 'TAKEDOWN_ATT_B',
       'SUB_ATT_R', 'SUB_ATT_B', 'REV_R', 'REV_B', 'CTRL_R', 'CTRL_B',
       'STR_HEAD_R', 'STR_HEAD_ATT_R', 'STR_HEAD_B', 'STR_HEAD_ATT_B',
       'STR_BODY_R', 'STR_BODY_ATT_R', 'STR_BODY_B', 'STR_BODY_ATT_B',
       'STR_LEG_R', 'STR_LEG_ATT_R', 'STR_LEG_B', 'STR_LEG_ATT_B',
       'DISTANCE_R', 'DISTANCE_ATT_R', 'DISTANCE_B', 'DISTANCE_ATT_B',
       'CLINCH_R', 'CLINCH_ATT_R', 'CLINCH_B', 'CLINCH_ATT_B', 'GROUND_R',
       'GROUND_ATT_R', 'GROUND_B', 'GROUND_ATT_B', 'LOCATION', 'DATE',
       'NAME_R', 'RECORD_R', 'HEIGHT_R', 'WEIGHT_R', 'REACH_R', 'STANCE_R',
       'DOB_R', 'SIG_STR_L_PM_R', 'SIG_STR_ACC_R', 'SIG_STR_ABS_PM_R',
 

### create feature "Age" for both competitors

In [27]:
from dateutil.parser import parse

df_fights["AGE_R"] = [None] * len(df_fights)
df_fights["AGE_B"] = [None] * len(df_fights)

# Calculate the age by the difference between the date of birth and the date of the fight
for i in range(len(df_fights)):
    try:
        df_fights.loc[i,"AGE_R"] = (parse(df_fights.loc[i,"DATE"]).year - parse(df_fights.loc[i,"DOB_R"]).year)
    except:
        pass
    try:
        df_fights.loc[i,"AGE_B"] = (parse(df_fights.loc[i,"DATE"]).year - parse(df_fights.loc[i,"DOB_B"]).year)
    except:
        pass

In [12]:
df_fights.shape

(6637, 91)

### create features for record before UFC
- therefore add the total stats and subtract the record made in the UFC later on 

In [13]:
RED = df_fights["RECORD_R"].apply(lambda x: x.replace("Record: ", "").split("-") if isinstance(x, str) else [np.nan] * 3)
BLUE = df_fights["RECORD_B"].apply(lambda x: x.replace("Record: ", "").split("-") if isinstance(x, str) else [np.nan] * 3)

RED_ARRAY = []
BLUE_ARRAY = []

for i,u in zip(RED, BLUE):
    RED_ARRAY.append(i)
    BLUE_ARRAY.append(u)

df_fights[["WIN_PRE_UFC_R", "LOSS_PRE_UFC_R", "DRAW_PRE_UFC_R"]] = pd.DataFrame(columns=["WIN_PRE_UFC_R", "LOSS_PRE_UFC_R", "DRAW_PRE_UFC_R"], data=RED_ARRAY, index=df_fights.index)
df_fights[["WIN_PRE_UFC_B", "LOSS_PRE_UFC_B", "DRAW_PRE_UFC_B"]] = pd.DataFrame(columns=["WIN_PRE_UFC_B", "LOSS_PRE_UFC_B", "DRAW_PRE_UFC_B"], data=BLUE_ARRAY, index=df_fights.index)

# Add No Contest to DRAWs
df_fights["DRAW_PRE_UFC_R"] = df_fights["DRAW_PRE_UFC_R"].apply(lambda x: int(x.replace(" NC)", "").split(" (")[0]) + 
                                                                            int(x.replace(" NC)", "").split(" (")[1])
                                                                            if x is not np.nan and "NC" in x else x)

df_fights["DRAW_PRE_UFC_B"] = df_fights["DRAW_PRE_UFC_B"].apply(lambda x: int(x.replace(" NC)", "").split(" (")[0]) + 
                                                                            int(x.replace(" NC)", "").split(" (")[1])
                                                                            if x is not np.nan and "NC" in x else x)


df_fights = df_fights.astype({"WIN_PRE_UFC_R":"float32", "LOSS_PRE_UFC_R":"float32", "DRAW_PRE_UFC_R":"float32", "WIN_PRE_UFC_B":"float32", "LOSS_PRE_UFC_B":"float", "DRAW_PRE_UFC_B":"float"})

df_fights[["WIN_PRE_UFC_R", "LOSS_PRE_UFC_R", "DRAW_PRE_UFC_R","WIN_PRE_UFC_B", "LOSS_PRE_UFC_B", "DRAW_PRE_UFC_B"]].dtypes

WIN_PRE_UFC_R     float32
LOSS_PRE_UFC_R    float32
DRAW_PRE_UFC_R    float32
WIN_PRE_UFC_B     float32
LOSS_PRE_UFC_B    float64
DRAW_PRE_UFC_B    float64
dtype: object

### Transform Data so each row represents accumulated statistics

In [15]:
df_fights['FIGHT_ID'] = np.arange(len(df_fights))
df_fights_b_to_r = df_fights.copy()

df_fights["ORIGINAL"] = 1
df_fights_b_to_r["ORIGINAL"] = 0

columns_swap = []
# reverse all fighter specific columns
for column in df_fights_b_to_r.columns:
    if column.endswith("_R"):
       column = column[:-2] + "_B"
    elif column.endswith("_B"):
       column = column[:-2] + "_R"
    else:
        pass
    columns_swap.append(column)

# reverse winner & loser
winners = []
for i in df_fights_b_to_r["WINNER"]:
    if i == "red":
        winner = "blue"
    elif i == "blue":
        winner = "red"
    else:
        winner = "draw/NC"
    winners.append(winner)
    
# Assign new columns
df_fights_b_to_r.columns = columns_swap
df_fights_b_to_r['WINNER'] = winners
# bring columns into the same order for Union
df_fights_rev_b = df_fights_b_to_r[df_fights.columns]
print(df_fights.shape, df_fights_rev_b.shape)
print(df_fights.isna().sum().sum(), df_fights_rev_b.isna().sum().sum())

(6637, 99) (6637, 99)
2224 2224


#### Union both dataframes and calculate accumulated statistics

In [16]:
df_fights_all = pd.concat([df_fights, df_fights_b_to_r]).sort_values("FIGHT_ID")

print(df_fights_all.shape)
print(df_fights_all.isna().sum().sum())

(13274, 99)
4448


In [17]:
df_fights_all = df_fights_all.sort_values("FIGHT_ID", ascending=False)

In [18]:
# We want to get the record of the time of the fight, but Winner is also our target variable
# copy, because get_dummies() makes the original column disappear
df_fights_all["WINNER_1"] = df_fights_all["WINNER"]

df_fights_all = pd.get_dummies(df_fights_all, columns=["WINNER_1", "WIN_METHOD", "ROUNDS"], prefix="", prefix_sep='')
df_fights_all = pd.get_dummies(df_fights_all, columns=["STANCE_R"], prefix="STANCE", prefix_sep='_')

df_fights_all = df_fights_all.rename(columns={"red":"WIN", "blue":"LOSS", "draw/NC":"DRAW", 
                            "Could Not Continue":"COULD_NOT_CON",
                            "Decision - Majority":"DEC_MAJORITY",
                            "Decision - Split":"DEC_SPLIT",
                            "Decision - Unanimous":"DEC_UNANIMOUS",
                            "KO/TKO":"KO_TKO", "Other":"OTHER",
                            "Overturned":"OVERTURNED", "Submission":"SUBMISSION",
                            "TKO - Doctor's Stoppage":"TKO_DOC_STOP",
                            "3 Rnd (5-5-5)":"3_ROUND", "5 Rnd (5-5-5-5-5)":"5_ROUND",
                            "STANCE_Open Stance":"OPEN_STANCE_R", "STANCE_Orthodox":"ORTHODOX_R",
                            "STANCE_Sideways":"SIDEWAYS_R", "STANCE_Southpaw":"SOUTHPAW_R",
                            "STANCE_Switch":"SWITCH_R"})


In [19]:
df_fights_all.replace(to_replace=[None], value=np.nan, inplace=True)
test = df_fights_all.astype({"WIN_PRE_UFC_R":"float32","LOSS_PRE_UFC_R":"float32", "DRAW_PRE_UFC_R" :"float32","WIN_PRE_UFC_B":"float32", "LOSS_PRE_UFC_B":"float32", "DRAW_PRE_UFC_B":"float32"})

## Win streak? Champion flag?

In [20]:
avg_columns = ["KD_R"
                ,"KD_B"
                ,"SIG_STR_R"
                ,"SIG_STR_ATT_R"
                ,"SIG_STR_B"
                ,"SIG_STR_ATT_B"
                ,"TOTAL_STR_R"
                ,"TOTAL_STR_ATT_R"
                ,"TOTAL_STR_B"
                ,"TOTAL_STR_ATT_B"
                ,"TAKEDOWN_R"
                ,"TAKEDOWN_ATT_R"
                ,"TAKEDOWN_B"
                ,"TAKEDOWN_ATT_B"
                ,"SUB_ATT_R"
                ,"SUB_ATT_B"
                ,"REV_R"
                ,"REV_B"
                ,"CTRL_R"
                ,"CTRL_B"
                ,"STR_HEAD_R"
                ,"STR_HEAD_ATT_R"
                ,"STR_HEAD_B"
                ,"STR_HEAD_ATT_B"
                ,"STR_BODY_R"
                ,"STR_BODY_ATT_R"
                ,"STR_BODY_B"
                ,"STR_BODY_ATT_B"
                ,"STR_LEG_R"
                ,"STR_LEG_ATT_R"
                ,"STR_LEG_B"
                ,"STR_LEG_ATT_B"
                ,"DISTANCE_R"
                ,"DISTANCE_ATT_R"
                ,"DISTANCE_B"
                ,"DISTANCE_ATT_B"
                ,"CLINCH_R"
                ,"CLINCH_ATT_R"
                ,"CLINCH_B"
                ,"CLINCH_ATT_B"
                ,"GROUND_R"
                ,"GROUND_ATT_R"
                ,"GROUND_B"
                ,"GROUND_ATT_B"
                ,"AVG_FIGHT_TIME"]

sum_columns = ["LOSS"
                ,"DRAW"
                ,"WIN"
                ,"COULD_NOT_CON"
                ,"DQ"
                ,"DEC_MAJORITY"
                ,"DEC_SPLIT"
                ,"DEC_UNANIMOUS"
                ,"KO_TKO"
                ,"OTHER"
                ,"OVERTURNED"
                ,"SUBMISSION"
                ,"TKO_DOC_STOP"
                ,"3_ROUND"
                ,"5_ROUND"]

In [21]:
# Function for calculating streaks
def generate_streak_info(df, column):
  
    data = df[f"{column}"].to_frame()
    data['start_of_streak'] = data[f"{column}"].ne(data[f"{column}"].shift())
    data['streak_id'] = data.start_of_streak.cumsum()
    data['streak_counter'] = data.groupby('streak_id').cumcount() + 1
    df_with_streaks = pd.concat([df, data['streak_counter']], axis=1)
    return df_with_streaks


In [32]:

output = []

# compute for each fighter aggregated statistics
for i in df_fights_all.FIGHTER_R.unique():

    df_fighter = df_fights_all[df_fights_all["FIGHTER_R"] == i].copy()


    # GET FEATURES IS_CHAMPION AND WAS_CHAMPION
    df_fighter["CHAMPION"] = df_fighter.apply(lambda x: 1 if x.TITLE_FIGHT == 1 and x.WINNER == "red" else 0, axis=1)
    df_fighter["still_CHAMPION"] = df_fighter.apply(lambda x: 1 if x.TITLE_FIGHT == 1 and x.WINNER == "draw/NC" else 0, axis=1)

    df_fighter["CHAMPION_roll_1"] = df_fighter["CHAMPION"].rolling(1).sum().shift(1)
    df_fighter["CHAMPION_roll_2"] = df_fighter["CHAMPION"].rolling(1).sum().shift(2)
    df_fighter["still_CHAMPION_roll_1"] = df_fighter["still_CHAMPION"].rolling(1).sum().shift(1)
    
    # if the fighter won his last champion ship fight then he is the champion.
    # if the champion pulls a draw he still is the champ
    
    df_fighter["IS_CHAMPION"] = df_fighter.apply(lambda x: 1 if x.CHAMPION_roll_1 == 1 
                                                        or (x.CHAMPION_roll_2 == 1 and x.still_CHAMPION_roll_1 == 1) 
                                                        else 0, axis=1)
    # if the fighter won a champion ship match and is not the champion currently then he was a champion
    df_fighter["WAS_CHAMPION"] = df_fighter["CHAMPION"].expanding().max().shift(1) - df_fighter["IS_CHAMPION"]
    
    # GET FEATURE WINNING STREAK: Winning streak is displayed as positive numbers and losses in negativ
    df_fighter["streak_counter"] = generate_streak_info(df_fighter, "WIN")["streak_counter"]
    df_fighter["WIN_LOSS_STREAK"] = df_fighter.apply(lambda x: x.streak_counter if x.WIN == 1 else 0, axis=1)
    df_fighter["WIN_LOSS_STREAK"] = df_fighter.apply(lambda x: -x.streak_counter if x.LOSS == 1 and x.WIN_LOSS_STREAK == 0 else x.WIN_LOSS_STREAK, axis=1)
    df_fighter["WIN_LOSS_STREAK"] = df_fighter["WIN_LOSS_STREAK"].shift(1)

    # shift() for only aggregating the infos of previous fights

    # avg_columns = list(test.columns[7:51])
    # avg_columns.append(test.columns[86])

    for avg_column in avg_columns:
        df_fighter[f"{avg_column}"] = df_fighter[f"{avg_column}"].expanding().mean().shift(1) 


    # sum_columns = list(test.columns[83:86])
    # for i in test.columns[97:]:
    #     sum_columns.append(i)

    for sum_column in sum_columns:
        df_fighter[f"{sum_column}"] = df_fighter[f"{sum_column}"].expanding().sum().shift(1) 

    # calculate stats before ufc debut
    df_fighter["WIN_PRE_UFC_R"] = df_fighter["WIN_PRE_UFC_R"] - df_fighter["WIN"]
    df_fighter["LOSS_PRE_UFC_R"] = df_fighter["LOSS_PRE_UFC_R"] - df_fighter["LOSS"]
    df_fighter["DRAW_PRE_UFC_R"] = df_fighter["WIN_PRE_UFC_R"] - df_fighter["DRAW"]     

    output.append(df_fighter)

# concat all fighters
df_collect = pd.concat(output).sort_index()

### Remove unwanted columns
- because in the next step the competitors will be joined again. The columns will represent the stats of the individual fighter and the stats of all opponents they had previously.

In [50]:
unwanted_columns = [ # Helper columns for calculating champion-features
                    "CHAMPION"
                    ,"still_CHAMPION"
                    ,"CHAMPION_roll_1"
                    ,"CHAMPION_roll_2"
                    ,"still_CHAMPION_roll_1"
                    ,"streak_counter"
                    # unnecessary columns
                    ,"REFEREE"                      # too many categories and they change with time
                    ,"LAST_ROUND"                   # redundant information with AVG_FIGHT_TIME
                    ,"LAST_ROUND_TIME"              # redundant information with AVG_FIGHT_TIME
                    ,"FIGHTER_B"                    # The tables will be joined and FIGHTER_R from the reversed rows becomes FIGHTER_B
                    ,"LOCATION"                     # Since there is no information about the fighter's location, this function has no use for displaying a possible travel time or time difference for each fighter
                    ,"DATE"                         # was used for the calculation of the age
                    ,"NAME_R"                       # redundant
                    ,"RECORD_R"                     # was used for extracting wins, losses and draws
                    ,"DOB_R"                        # was used for the calculation of the age
                    ,"SIG_STR_L_PM_R"               #
                    ,"SIG_STR_ACC_R"                #
                    ,"SIG_STR_ABS_PM_R"             #
                    ,"SIG_STR_DEF_R"                # 
                    ,"AVG_TAKEDOWN_15_MIN_R"        # 
                    ,"TAKEDOWN_ACC_R"               #
                    ,"TAKEDOWN_DEF_R"               #
                    ,"AVG_SUB_ATT_15_MIN_R"         # Those values are calculated across the hole career and cannot be use for model training 
                    ,"NAME_B"
                    ,"RECORD_B"                     # same for the "_B"-columns
                    ,"HEIGHT_B"
                    ,"REACH_B"
                    ,"STANCE_B"
                    ,"DOB_B"
                    ,"SIG_STR_L_PM_B"
                    ,"SIG_STR_ACC_B"
                    ,"SIG_STR_ABS_PM_B"
                    ,"SIG_STR_DEF_B"
                    ,"AVG_TAKEDOWN_15_MIN_B"
                    ,"TAKEDOWN_ACC_B"
                    ,"TAKEDOWN_DEF_B"
                    ,"AVG_SUB_ATT_15_MIN_B"
                    ,"AGE_B"
                    ,"WIN_PRE_UFC_B"
                    ,"LOSS_PRE_UFC_B"
                    ,"DRAW_PRE_UFC_B"
                    ,"STANCE_B"]               

df_collect_1 = df_collect.drop(unwanted_columns, axis='columns')

In [66]:
# join the stats of the competetors in one row
# The stats of all previous opponents well be displayed with the suffix "_OPP_" before "_R" or "_B"

df_original = df_collect_1[df_collect_1["ORIGINAL"]==1]
df_reversed = df_collect_1[df_collect_1["ORIGINAL"]==0]

# drop shared stats of the fight
df_reversed = df_reversed.drop(["BOUT", "WINNER"], axis='columns')


columns_swap_original = []
# reverse all fighter specific columns
for column in df_original.columns:
    if column.endswith("_B"):
       column = column[:-2] + "_OPP_R"
    elif column.endswith("_R"):
        pass
    elif column in ["BOUT", "WINNER", "FIGHT_ID"]:
        pass
    else:  
        column = column + "_R"
    columns_swap_original.append(column)


columns_swap_switched = []
# reverse all fighter specific columns
for column in df_reversed.columns:
    if column.endswith("_B"):
       column = column[:-2] + "_OPP_B"
    elif column.endswith("_R"):
       column = column[:-2] + "_B" 
    elif column == "FIGHT_ID":
        pass
    else:
        column = column + "_B"
    columns_swap_switched.append(column)

df_original.columns = columns_swap_original
df_reversed.columns = columns_swap_switched

In [67]:
df_fights_complete = df_original.merge(df_reversed, on="FIGHT_ID", how="inner", suffixes=('_x', '_y'))
df_fights_complete = df_fights_complete.drop(columns = ["FIGHT_ID", "ORIGINAL_R", "ORIGINAL_B"])
df_fights_complete = df_fights_complete.reindex(sorted(df_fights_complete.columns), axis=1)

In [68]:
df_fights_complete["BOUT"].unique()

array(['Light Heavyweight Bout', 'Heavyweight Bout', 'Welterweight Bout',
       'Flyweight Bout', "Women's Flyweight Bout", 'Bantamweight Bout',
       "Women's Strawweight Bout", 'UFC Middleweight Title Bout',
       "UFC Women's Strawweight Title Bout", 'Lightweight Bout',
       'Middleweight Bout', 'Featherweight Bout',
       "Women's Bantamweight Bout", 'UFC Lightweight Title Bout',
       'UFC Bantamweight Title Bout', 'Catch Weight Bout',
       "Women's Featherweight Bout", 'UFC Welterweight Title Bout',
       "UFC Women's Bantamweight Title Bout",
       'UFC Featherweight Title Bout', 'UFC Light Heavyweight Title Bout',
       "UFC Women's Flyweight Title Bout", 'UFC Heavyweight Title Bout',
       'UFC Flyweight Title Bout', 'UFC Interim Bantamweight Title Bout',
       'UFC Interim Heavyweight Title Bout',
       "UFC Women's Featherweight Title Bout",
       'UFC Interim Lightweight Title Bout',
       'UFC Interim Middleweight Title Bout',
       'UFC Interim Welterwei

### Extract weight from the competition
- the individual weight is the current weight and may have changed by the carrier.
- If no weight is available or the catch weight is displayed, the fighter's statistics are used.

In [69]:
df_fights_complete["BOUT"] = df_fights_complete["BOUT"].replace({'Light Heavyweight Bout': '205'
                                                                ,'Heavyweight Bout':'265'
                                                                ,'Welterweight Bout':'170'
                                                                ,'Flyweight Bout':'125'
                                                                ,"Women's Flyweight Bout":'125'
                                                                ,"Bantamweight Bout":"135"
                                                                ,"Women's Strawweight Bout":"115"
                                                                ,"UFC Middleweight Title Bout":"185"
                                                                ,"UFC Women's Strawweight Title Bout":"115"
                                                                ,"Lightweight Bout":"155 "
                                                                ,"Middleweight Bout":"185"
                                                                ,"Featherweight Bout":"145"
                                                                ,"Women's Bantamweight Bout":"135"
                                                                ,"UFC Lightweight Title Bout":"155"
                                                                ,"UFC Bantamweight Title Bout":"135"
                                                                ,"Catch Weight Bout":None
                                                                ,"Women's Featherweight Bout":"145"
                                                                ,"UFC Welterweight Title Bout":"170"
                                                                ,"UFC Women's Bantamweight Title Bout":"135"
                                                                ,"UFC Featherweight Title Bout":"145"
                                                                ,"UFC Light Heavyweight Title Bout":"205"
                                                                ,"UFC Women's Flyweight Title Bout":"125"
                                                                ,"UFC Heavyweight Title Bout":"265"
                                                                ,"UFC Flyweight Title Bout":"125"
                                                                ,"UFC Interim Bantamweight Title Bout":"135"
                                                                ,"UFC Interim Heavyweight Title Bout":"265"
                                                                ,"UFC Women's Featherweight Title Bout":"145"
                                                                ,"UFC Interim Lightweight Title Bout":"155"
                                                                ,"UFC Interim Middleweight Title Bout":"185"
                                                                ,"UFC Interim Welterweight Title Bout":"170"
                                                                ,"UFC Interim Featherweight Title Bout":"175"
                                                                ,"UFC Interim Light Heavyweight Title Bout":"205"
                                                                })

df_fights_complete["BOUT"] = df_fights_complete.apply(lambda x: x.WEIGHT_R.replace(" lbs.","") if x.BOUT is None else x.BOUT, axis=1).astype("float32")
df_fights_complete["BOUT"] = df_fights_complete.apply(lambda x: x.WEIGHT_B.replace(" lbs.","") if x.BOUT is None else x.BOUT, axis=1).astype("float32")

# Indivdual no longer needed
df_fights_complete = df_fights_complete.drop(["WEIGHT_R", "WEIGHT_B","WEIGHT_OPP_R", "WEIGHT_OPP_B"], axis=1)

df_fights_complete["BOUT"].unique()

array([205., 265., 170., 125., 135., 115., 185., 155., 145., 175., 235.],
      dtype=float32)

In [70]:
shape = df_fights_complete.shape
nan = np.sum(df_fights_complete.isnull())
dublicates = np.sum(df_fights_complete.duplicated())

print('shape: ', shape)
print('NaNs: ', nan[nan > 0], sep="\n")
print('Dublicates: ', dublicates)

shape:  (6637, 158)
NaNs: 
3_ROUND_B            1419
3_ROUND_R             732
5_ROUND_B            1419
5_ROUND_R             732
AGE_B                 135
                     ... 
WIN_LOSS_STREAK_B    1419
WIN_LOSS_STREAK_R     732
WIN_PRE_UFC_B        1444
WIN_PRE_UFC_R         754
WIN_R                 732
Length: 136, dtype: int64
Dublicates:  0


In [71]:
# Remove rows where more than ~ 30 percent of columns are nan
df_fights_complete.replace(to_replace=[None], value=np.nan, inplace=True)
df_fights_complete = df_fights_complete.dropna(thresh=len(df_fights_complete.columns)/1.4)
nan_cols = df_fights_complete.columns[df_fights_complete.isnull().any()]

# impute missing values using the median
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(df_fights_complete[nan_cols])
df_fights_complete.loc[:,nan_cols] = imp.transform(df_fights_complete[nan_cols])

# all dtypes except "WINNER", "FIHGTER_R" & "FIHGTER_B" have a numeric format
df_fights_complete.to_csv("./output/fights_cleaned.csv", sep=';', index=False)